In [1]:
from torch import nn
from torch.nn.parameter import Parameter
import torch.nn.functional as F
import torch
import math

from skorch import NeuralNetClassifier
from skorch.callbacks import LRScheduler, BatchScoring, Checkpoint, LoadInitState
from sklearn.datasets import make_classification

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

from sklearn.svm import SVC
from sklearn.compose import make_column_transformer, make_column_selector
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import matthews_corrcoef, make_scorer, classification_report
import xgboost as xg

warnings.filterwarnings("ignore")
np.set_printoptions(precision=3)
pd.set_option('precision', 3)

In C:\Users\ilyac\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The text.latex.preview rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In C:\Users\ilyac\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The mathtext.fallback_to_cm rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In C:\Users\ilyac\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: Support for setting the 'mathtext.fallback_to_cm' rcParam is deprecated since 3.3 and will be removed two minor releases later; use 'mathtext.fallback : 'cm' instead.
In C:\Users\ilyac\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The validate_bool_maybe_none function was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In C:\Users\ilyac\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mpl

### Load the dataset

In [2]:
data = pd.read_csv('train.csv', index_col=0)
data.TotalCharges = data.TotalCharges.replace(' ', 0).astype(float)
data = data.drop(['Gender', 'Phone', 'MultiplePhones'], axis=1)
data = data.replace({'Senior': {1: 'Yes', 0: 'No'}})
data.head(1)

,Senior,Married,Dependents,Tenure,Internet,Security,Backup,Insurance,Support,TV,Movies,Contract,EBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
Index,,,,,,,,,,,,,,,,,
0,No,Yes,Yes,67,DSL,Yes,Yes,Yes,Yes,Yes,No,Two year,No,Credit card (automatic),54.2,3623.95,False


### Split it

In [3]:
# X_train, X_test, Y_train, Y_test = train_test_split(data.drop(['Churn'], axis=1), 
#                                                     data.Churn,
#                                                     test_size = 0.4)

X_train, Y_train = data.drop(['Churn'], axis=1), data.Churn
# X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size=0.1)

### Create a preprocessor

In [4]:
preprocessor = make_column_transformer(
    (StandardScaler(), make_column_selector(dtype_include=np.number)),
    (OneHotEncoder(), make_column_selector(dtype_include=object))
)

In [5]:
X_train_preprocessed = preprocessor.fit_transform(X_train).astype(np.float32)
Y_train = np.array(Y_train).astype(np.int64)
# X_test_preprocessed = preprocessor.transform(X_test).astype(np.float32)
# Y_test = np.array(Y_test).astype(np.int64)

### Создаем сеть на торче и используем ее в sklearn через skorch

Используем arcface из задачи face recognition, который можно применять для open set classification. По сути, на важно, что он разделяет вектора фичей в пространстве так, чтобы они лежали на гиперсфере и вектора разных классов были как можно дальше друг от друга, в то время как вектора одного и того же класса как можно ближе друг к другу в плане углового расстояния

In [6]:
# Размер вектора фичей на выходе сети
#
embed_feat_size = 16

# L2 normalization layer.
#
def l2n(x, eps=1e-6):
    return x / (torch.norm(x, p=2, dim=1, keepdim=True) + eps).expand_as(x)

class L2N(nn.Module):

    def __init__(self, eps=1e-6):
        super(L2N,self).__init__()
        self.eps = eps

    def forward(self, x):
        return l2n(x, eps=self.eps)


# ArcFace head.
#
class ArcMarginProduct(nn.Module):
    r"""Implementation of large margin arc distance: :
        Args:
            in_features: size of each input sample
            out_features: size of each output sample (should match number of classes in the training dataset)
            s: norm of input feature (hypersphere radius)
            m: margin
            cos(theta + m)
        """
    def __init__(self, in_features=embed_feat_size, out_features=2, s=20.0, m=.6, easy_margin=False):
        super(ArcMarginProduct, self).__init__()
        self.criterion = nn.CrossEntropyLoss()
        self.in_features = in_features
        self.out_features = out_features
        self.s = s
        self.m = m
        self.weight = Parameter(torch.FloatTensor(out_features, in_features))
        nn.init.xavier_uniform_(self.weight)

        self.easy_margin = easy_margin
        self.cos_m = math.cos(m)
        self.sin_m = math.sin(m)
        self.th = math.cos(math.pi - m)
        self.mm = math.sin(math.pi - m) * m

    def forward(self, input, target):
        # --------------------------- cos(theta) & phi(theta) ---------------------------
        cosine = F.linear(F.normalize(input), F.normalize(self.weight))
        sine = torch.sqrt((1.0 - torch.pow(cosine, 2)).clamp(0, 1))
        phi = cosine * self.cos_m - sine * self.sin_m
        if self.easy_margin:
            phi = torch.where(cosine > 0, phi, cosine)
        else:
            phi = torch.where(cosine > self.th, phi, cosine - self.mm)
        
        one_hot = torch.zeros(cosine.size(), device='cpu')
        
        one_hot.scatter_(1, target.view(-1, 1).long(), 1)
        output = (one_hot * phi) + ((1.0 - one_hot) * cosine) 
        output *= self.s
        # print(output)

        return self.criterion.forward(output, target)

    
class ArcFaceNet(nn.Module):
    def __init__(self):
        super(ArcFaceNet, self).__init__()
        self.net = MyNet()

    def forward(self, x):
        x = self.net(x)
        return x
    
    
class MyNet(nn.Module):
    def __init__(self, input_size=39, embedding_size=embed_feat_size * 2):
        super(MyNet, self).__init__()
        self.norm = L2N()
        
        self.fc1 = nn.Linear(input_size, embedding_size)
        self.bn1 = nn.BatchNorm1d(embedding_size)
        self.relu1 = nn.PReLU(embedding_size)
        
        self.fc2 = nn.Linear(embedding_size, embedding_size // 2)
        self.bn2 = nn.BatchNorm1d(embedding_size // 2)
        self.relu2 = nn.PReLU(embedding_size // 2)
        
    def forward(self, x):
        x = self.relu1(self.bn1(self.fc1(x)))
        x = self.relu2(self.bn2(self.fc2(x)))
        x = self.norm(x)
        
        return x

In [7]:
embeddingNet = ArcFaceNet
criterion = ArcMarginProduct

cp = Checkpoint(dirname='checkpoints6')
load_state = LoadInitState(cp)
    
net = NeuralNetClassifier(
    embeddingNet,
    max_epochs=200,
    lr=3e-4,
    criterion=criterion,
    optimizer=torch.optim.AdamW,
    optimizer__amsgrad=True,
    optimizer__weight_decay=0.25,
    # Shuffle training data on each epoch
    iterator_train__shuffle=True,
    callbacks=[
        ('lr_scheduler',
         LRScheduler(policy=torch.optim.lr_scheduler.StepLR,
                     gamma=0.4,
                     step_size=25)),
        cp,
        load_state,
    ],
)

In [8]:
net.fit(X_train_preprocessed, Y_train)

  epoch    train_loss    valid_acc    valid_loss    cp     dur
-------  ------------  -----------  ------------  ----  ------
      1       10.8427       0.0362        9.3915     +  0.1965
      2        8.3707       0.0431        7.2482     +  0.1287
      3        6.9463       0.0376        6.3072     +  0.1227
      4        6.2771       0.0195        5.8821     +  0.1207
      5        5.8681       0.0153        5.5223     +  0.1197
      6        5.5664       0.0139        5.2466     +  0.1227
      7        5.3523       0.0111        5.1219     +  0.1366
      8        5.1523       0.0056        4.9066     +  0.1466
      9        4.9760       0.0056        4.8313     +  0.1217
     10        4.8541       0.0056        4.6786     +  0.1217
     11        4.7085       0.0042        4.6033     +  0.1406
     12        4.5791       0.0042        4.4885     +  0.1356
     13        4.4954       0.0042        4.3894     +  0.1277
     14        4.4181       0.0042        4.2760     + 

<class 'skorch.classifier.NeuralNetClassifier'>[initialized](
  module_=ArcFaceNet(
    (net): MyNet(
      (norm): L2N()
      (fc1): Linear(in_features=39, out_features=32, bias=True)
      (bn1): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu1): PReLU(num_parameters=32)
      (fc2): Linear(in_features=32, out_features=16, bias=True)
      (bn2): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu2): PReLU(num_parameters=16)
    )
  ),
)

### Train SVM on new features

We can use linear kernel as two classes already separated on the hypersphere

In [11]:
X_train_features = net.forward(X_train_preprocessed)
clf = SVC(kernel='linear', class_weight='balanced', C=0.1)

In [12]:
clf.fit(X_train_features, Y_train)

SVC(C=0.1, class_weight='balanced', kernel='linear')

### Calculate matthews coefficient on our test data

In [13]:
# X_test_features = net.forward(X_test_preprocessed)
# y_pred = clf.predict(X_test_features)
# print("matthews test ", matthews_corrcoef(Y_test, y_pred))
# print(classification_report(Y_test, y_pred))

In [14]:
X_train_features = net.forward(X_train_preprocessed)
y_pred_train = clf.predict(X_train_features)
print("matthews train ", matthews_corrcoef(Y_train, y_pred_train))
print(classification_report(Y_train, y_pred_train))

matthews train  0.49966470531820584
              precision    recall  f1-score   support

           0       0.90      0.79      0.84      2639
           1       0.56      0.75      0.64       952

    accuracy                           0.78      3591
   macro avg       0.73      0.77      0.74      3591
weighted avg       0.81      0.78      0.79      3591



### Get results on test data

In [15]:
data_test = pd.read_csv('test.csv', index_col=0)
data_test.TotalCharges = data_test.TotalCharges.replace(' ', 0).astype(float)
data_test = data_test.drop(['Gender', 'Phone', 'MultiplePhones'], axis=1)
data_test = data_test.replace({'Senior': {1: 'Yes', 0: 'No'}})
data_test.head(1)

,Senior,Married,Dependents,Tenure,Internet,Security,Backup,Insurance,Support,TV,Movies,Contract,EBilling,PaymentMethod,MonthlyCharges,TotalCharges
Index,,,,,,,,,,,,,,,,
0,No,No,No,58,Fiber optic,Yes,Yes,No,No,Yes,Yes,One year,No,Bank transfer (automatic),99.15,5720.95


### Preprocess test data

In [16]:
X_test_preprocessed = preprocessor.transform(data_test).astype(np.float32)

### Save answers

In [20]:
X_test_features = net.forward(X_test_preprocessed)
y_test_pred = clf.predict(X_test_features) == True
answer = pd.DataFrame(data=y_test_pred,
                      columns=['Churn'])
answer.index.name = 'Index'

In [21]:
answer.to_csv('./answer14.csv')

In [36]:
data2 = pd.read_csv('answer14.csv', index_col=0)

In [38]:
data2.value_counts()

Churn
False    1573
True      822
dtype: int64

### Смотрим соотношение true к false

In [37]:
data2.value_counts()[1] / data2.value_counts()[0]

0.5225683407501589